The model described here is a Convolutional Autoencoder (CAE) combined with a Gaussian Mixture Model (GMM) for the purpose of anomaly detection in recordings. The process can be broken down into several key steps:

Feature Extraction:
The extract_features_and_train function processes pairs of recordings (acoustic emission (AE) and electrical current (EC) recordings) by segmenting them and then calculating specific features (like variance and energy) for each segment pair.
These features are prepared for input into the CAE.

Convolutional Autoencoder (CAE):
The CAE is defined in the ConvAutoencoder class, which includes both an encoder and a decoder.
Encoder: Compresses the input features into a lower-dimensional representation using two convolutional layers, each followed by a ReLU activation function.
Decoder: Attempts to reconstruct the input from the encoded representation using two transposed convolutional layers (also known as deconvolutional layers), each followed by a ReLU activation function, with the final layer using a Sigmoid activation to output the reconstructed features.
The CAE is trained to minimize the reconstruction error, encouraging the encoder to learn a useful representation of the input features.

Training the CAE:
The extracted features are converted into a PyTorch tensor and used as input to the CAE.
The CAE is trained using the Mean Squared Error (MSE) loss between the output (reconstructed features) and the input features, with the Adam optimizer used for optimization.
After training, the encoder part of the CAE is used to encode the features into a compressed representation.

Gaussian Mixture Model (GMM):
The encoded features from the CAE's encoder are then used to train a GMM.
The GMM is a probabilistic model that assumes all the data points are generated from a mixture of a finite number of Gaussian distributions with unknown parameters.
The GMM is trained to fit the encoded features, effectively learning to model the distribution of the normal operational data.

Anomaly Detection:
After training, the GMM can be used to classify new recordings as either normal or anomalous.
This classification is based on how well the features of a new recording fit the learned Gaussian distributions. Recordings that fit poorly (e.g., have low likelihood under the model) can be considered anomalous.
This approach leverages the CAE's ability to learn compact and relevant representations of the data, which are then used by the GMM to effectively model the normal operational state. Anomalies are detected as deviations from this learned normal state.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###**Debugged code (290624) for a more reasonable number of segments (401)**

Segments in the 97.5th percentile of the learnt distribution based on normal recordings marked as anomalous, then proportion of anomalous segments in each recording measured and recordings with proportions higher than 99% of normal recordings are flagged as anomalous

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
import joblib
import scipy.stats as stats
import time

# Constants
AE_SAMPLING_RATE = 2000 * 1000  # 2000 kHz
EC_SAMPLING_RATE = 100 * 1000   # 100 kHz
SEGMENT_DURATION_MS = 50  # Segment length in milliseconds
ANOMALY_THRESHOLD_PERCENTILE = 2.5  # Threshold percentile for segment anomaly (beyond 97.5th percentile = anomalous)
RECORDING_ANOMALY_THRESHOLD = 99  # Threshold for recording anomaly (beyond 99th percentile = anomalous)

def load_training_files(directory):
    train_files = []
    for root, dirs, files in os.walk(directory):
        if '/raw' in root or '\\raw' in root:  # Adjusted for both Unix and Windows paths
            ae_file = os.path.join(root, 'Sampling2000KHz_AEKi-0.parquet')
            ec_file = os.path.join(root, 'Sampling100KHz_Irms_Grinding-Grinding spindle current L1-Grinding spindle current L2-Grinding spindle current L3-0.parquet')
            if os.path.exists(ae_file) and os.path.exists(ec_file):
                train_files.append((ae_file, ec_file))
    print(f"Train files: {train_files}")
    return train_files

def load_test_files(directory):
    test_files = []
    for root, dirs, files in os.walk(directory):
        if '/raw' in root or '\\raw' in root:  # Adjusted for both Unix and Windows paths
            ae_file = os.path.join(root, 'Sampling2000KHz_AEKi-0.parquet')
            ec_file = os.path.join(root, 'Sampling100KHz_Irms_Grinding-Grinding spindle current L1-Grinding spindle current L2-Grinding spindle current L3-0.parquet')
            if os.path.exists(ae_file) and os.path.exists(ec_file) and "OK" in root:
                test_files.append((root, ae_file, ec_file))
    print(f"Test files: {test_files}")
    return test_files

def load_recording(file_path):
    try:
        df = pd.read_parquet(file_path)
        print(f"Loaded {file_path} with shape {df.shape}")
        recording = df.iloc[:, 0].values
        return recording
    except Exception as e:
        print(f"Failed to load {file_path}: {e}")
        return np.array([])

def calculate_segment_size(sampling_rate, segment_duration_ms):
    return int(sampling_rate * segment_duration_ms / 1000)

def load_and_segment_recordings(ae_file, ec_file, segment_duration_ms, pad_value=0):
    ae_recording = load_recording(ae_file)
    ec_recording = load_recording(ec_file)

    segment_size_ae = calculate_segment_size(AE_SAMPLING_RATE, segment_duration_ms)
    segment_size_ec = calculate_segment_size(EC_SAMPLING_RATE, segment_duration_ms)

    def segment_and_pad(recording, segment_size):
        segments = [recording[i:i+segment_size] for i in range(0, len(recording), segment_size)]
        padded_segments = [np.pad(segment, (0, max(0, segment_size - len(segment))), 'constant', constant_values=pad_value) for segment in segments]
        print(f"Segment lengths: {[len(seg) for seg in padded_segments]}")
        return padded_segments

    ae_segments = segment_and_pad(ae_recording, segment_size_ae)
    ec_segments = segment_and_pad(ec_recording, segment_size_ec)
    return ae_segments, ec_segments

def calculate_features(segment):
    variance = np.var(segment)
    energy = np.sum(np.square(segment))
    return variance, energy

def calculate_features_for_pair(ae_segment, ec_segment):
    ae_variance, ae_energy = calculate_features(ae_segment)
    ec_variance, ec_energy = calculate_features(ec_segment)
    #print(f"Features: AE Variance {ae_variance}, AE Energy {ae_energy}, EC Variance {ec_variance}, EC Energy {ec_energy}")
    return ae_variance, ae_energy, ec_variance, ec_energy

class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d(1, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv1d(16, 4, 3, stride=2, padding=1),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(4, 16, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose1d(16, 1, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

def extract_features_and_train(train_files, segment_duration_ms):
    start_time = time.time()  # Record start time
    all_features = []
    for ae_file, ec_file in train_files:
        ae_segments, ec_segments = load_and_segment_recordings(ae_file, ec_file, segment_duration_ms)
        for ae_segment, ec_segment in zip(ae_segments, ec_segments):
            features = calculate_features_for_pair(ae_segment, ec_segment)
            all_features.append(features)
    all_features = np.array(all_features)

    if len(all_features) == 0:
        print("Error: all_features is empty. Please check data extraction.")
        return None, None

    features_tensor = torch.tensor(all_features, dtype=torch.float32).unsqueeze(1)

    if features_tensor.shape[0] == 0 or features_tensor.shape[1] != 1:
        print("Error: features_tensor is incorrectly shaped.")
        return None, None

    print("Shape of features_tensor:", features_tensor.shape)

    autoencoder = ConvAutoencoder()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

    num_epochs = 10
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = autoencoder(features_tensor)
        loss = criterion(outputs, features_tensor)
        loss.backward()
        optimizer.step()

    encoded_features = autoencoder.encoder(features_tensor).detach()

    encoded_features_2d = encoded_features.view(encoded_features.size(0), -1).numpy()

    gmm = GaussianMixture(n_components=3, covariance_type='full', random_state=0).fit(encoded_features_2d)

    # Determine the 97.5th percentile threshold for anomaly detection
    threshold = np.percentile(gmm.score_samples(encoded_features_2d), ANOMALY_THRESHOLD_PERCENTILE)
    joblib.dump((gmm, threshold), 'cae_gmm_model.pkl')

    end_time = time.time()  # Record end time
    print(f"Training time: {end_time - start_time} seconds")  # Print training duration

    return autoencoder, gmm, threshold

def detect_anomalies(autoencoder, gmm, threshold, ae_file, ec_file, segment_duration_ms):
    ae_segments, ec_segments = load_and_segment_recordings(ae_file, ec_file, segment_duration_ms)
    all_features = []

    start_time = time.time()  # Record start time for inference

    for ae_segment, ec_segment in zip(ae_segments, ec_segments):
        features = calculate_features_for_pair(ae_segment, ec_segment)
        all_features.append(features)
    all_features = np.array(all_features)

    if len(all_features) == 0:
        print("Error: all_features is empty. Please check data extraction.")
        return None

    features_tensor = torch.tensor(all_features, dtype=torch.float32).unsqueeze(1)
    encoded_features = autoencoder.encoder(features_tensor).detach()
    encoded_features_2d = encoded_features.view(encoded_features.size(0), -1).numpy()

    # Identify anomalous segments
    scores = gmm.score_samples(encoded_features_2d)
    anomalous_segments = scores < threshold

    # Calculate the proportion of anomalous segments
    proportion_anomalous = np.sum(anomalous_segments) / len(anomalous_segments)
    print(f"Proportion of anomalous segments: {proportion_anomalous:.4f}")

    return proportion_anomalous

def flag_recording_as_anomalous(proportion_anomalous, normal_proportions):
    # Determine the threshold for recording-level anomalies
    recording_threshold = np.percentile(normal_proportions, RECORDING_ANOMALY_THRESHOLD)
    is_anomalous = proportion_anomalous > recording_threshold
    return is_anomalous

# Define the directory containing your training data
training_data_directory = "/content/drive/MyDrive/Colab Notebooks/OK_test_6"

# Define the desired segment duration in milliseconds
segment_duration_ms = 50

# Load training and testing file paths
train_files = load_training_files(training_data_directory)

# Train the model
autoencoder, gmm, threshold = extract_features_and_train(train_files, segment_duration_ms)

# Calculate the proportion of anomalous segments in the normal training data
normal_proportions = []
for ae_file, ec_file in train_files:
    proportion_anomalous = detect_anomalies(autoencoder, gmm, threshold, ae_file, ec_file, segment_duration_ms)
    normal_proportions.append(proportion_anomalous)

# Save the normal proportions for later comparison
np.save('normal_proportions.npy', normal_proportions)

# Load NOK test files and flag recordings as anomalous if needed
test_data_directory = "/content/drive/MyDrive/Colab Notebooks/NOK_Measurements kopie"

test_files = load_test_files(test_data_directory)
for _, ae_file, ec_file in test_files:
    proportion_anomalous = detect_anomalies(autoencoder, gmm, threshold, ae_file, ec_file, segment_duration_ms)
    is_anomalous = flag_recording_as_anomalous(proportion_anomalous, normal_proportions)
    print(f"Recording {ae_file} is {'anomalous' if is_anomalous else 'normal'}")


Train files: [('/content/drive/MyDrive/Colab Notebooks/OK_test_6/2024.02.20_14.28.38_Grinding/raw/Sampling2000KHz_AEKi-0.parquet', '/content/drive/MyDrive/Colab Notebooks/OK_test_6/2024.02.20_14.28.38_Grinding/raw/Sampling100KHz_Irms_Grinding-Grinding spindle current L1-Grinding spindle current L2-Grinding spindle current L3-0.parquet'), ('/content/drive/MyDrive/Colab Notebooks/OK_test_6/2024.02.20_14.43.14_Grinding/raw/Sampling2000KHz_AEKi-0.parquet', '/content/drive/MyDrive/Colab Notebooks/OK_test_6/2024.02.20_14.43.14_Grinding/raw/Sampling100KHz_Irms_Grinding-Grinding spindle current L1-Grinding spindle current L2-Grinding spindle current L3-0.parquet'), ('/content/drive/MyDrive/Colab Notebooks/OK_test_6/2024.02.20_14.42.43_Grinding/raw/Sampling2000KHz_AEKi-0.parquet', '/content/drive/MyDrive/Colab Notebooks/OK_test_6/2024.02.20_14.42.43_Grinding/raw/Sampling100KHz_Irms_Grinding-Grinding spindle current L1-Grinding spindle current L2-Grinding spindle current L3-0.parquet'), ('/conte

In [ ]:
# Run trained model on OK test data
# Load test files and flag recordings as anomalous if needed
test_data_directory = "/content/drive/MyDrive/Colab Notebooks/OK_Measurements kopie"

test_files = load_test_files(test_data_directory)
for _, ae_file, ec_file in test_files:
    proportion_anomalous = detect_anomalies(autoencoder, gmm, threshold, ae_file, ec_file, segment_duration_ms)
    is_anomalous = flag_recording_as_anomalous(proportion_anomalous, normal_proportions)
    print(f"Recording {ae_file} is {'anomalous' if is_anomalous else 'normal'}")

Test files: [('/content/drive/MyDrive/Colab Notebooks/OK_Measurements kopie/2024.02.14_22.00.10_Grinding/raw', '/content/drive/MyDrive/Colab Notebooks/OK_Measurements kopie/2024.02.14_22.00.10_Grinding/raw/Sampling2000KHz_AEKi-0.parquet', '/content/drive/MyDrive/Colab Notebooks/OK_Measurements kopie/2024.02.14_22.00.10_Grinding/raw/Sampling100KHz_Irms_Grinding-Grinding spindle current L1-Grinding spindle current L2-Grinding spindle current L3-0.parquet'), ('/content/drive/MyDrive/Colab Notebooks/OK_Measurements kopie/2024.02.14_22.00.10_Grinding/2024.02.14_22.01.41_Grinding/raw', '/content/drive/MyDrive/Colab Notebooks/OK_Measurements kopie/2024.02.14_22.00.10_Grinding/2024.02.14_22.01.41_Grinding/raw/Sampling2000KHz_AEKi-0.parquet', '/content/drive/MyDrive/Colab Notebooks/OK_Measurements kopie/2024.02.14_22.00.10_Grinding/2024.02.14_22.01.41_Grinding/raw/Sampling100KHz_Irms_Grinding-Grinding spindle current L1-Grinding spindle current L2-Grinding spindle current L3-0.parquet'), ('/con

###**Optimisation for GPU use**

In [3]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
import joblib
import scipy.stats as stats
import time

# At the beginning of the script, ensure PyTorch is using CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Constants
AE_SAMPLING_RATE = 2000 * 1000  # 2000 kHz
EC_SAMPLING_RATE = 100 * 1000   # 100 kHz
SEGMENT_DURATION_MS = 50  # Segment length in milliseconds
ANOMALY_THRESHOLD_PERCENTILE = 2.5  # Threshold percentile for segment anomaly (beyond 97.5th percentile = anomalous)
RECORDING_ANOMALY_THRESHOLD = 99  # Threshold for recording anomaly (beyond 99th percentile = anomalous)

def load_training_files(directory):
    train_files = []
    for root, dirs, files in os.walk(directory):
        if '/raw' in root or '\\raw' in root:  # Adjusted for both Unix and Windows paths
            ae_file = os.path.join(root, 'Sampling2000KHz_AEKi-0.parquet')
            ec_file = os.path.join(root, 'Sampling100KHz_Irms_Grinding-Grinding spindle current L1-Grinding spindle current L2-Grinding spindle current L3-0.parquet')
            if os.path.exists(ae_file) and os.path.exists(ec_file):
                train_files.append((ae_file, ec_file))
    print(f"Train files: {train_files}")
    return train_files

def load_test_files(directory):
    test_files = []
    for root, dirs, files in os.walk(directory):
        if '/raw' in root or '\\raw' in root:  # Adjusted for both Unix and Windows paths
            ae_file = os.path.join(root, 'Sampling2000KHz_AEKi-0.parquet')
            ec_file = os.path.join(root, 'Sampling100KHz_Irms_Grinding-Grinding spindle current L1-Grinding spindle current L2-Grinding spindle current L3-0.parquet')
            if os.path.exists(ae_file) and os.path.exists(ec_file) and "OK" in root:
                test_files.append((root, ae_file, ec_file))
    print(f"Test files: {test_files}")
    return test_files

def load_recording(file_path):
    try:
        df = pd.read_parquet(file_path)
        print(f"Loaded {file_path} with shape {df.shape}")
        recording = df.iloc[:, 0].values
        return recording
    except Exception as e:
        print(f"Failed to load {file_path}: {e}")
        return np.array([])

def calculate_segment_size(sampling_rate, segment_duration_ms):
    return int(sampling_rate * segment_duration_ms / 1000)

def load_and_segment_recordings(ae_file, ec_file, segment_duration_ms, pad_value=0):
    ae_recording = load_recording(ae_file)
    ec_recording = load_recording(ec_file)

    segment_size_ae = calculate_segment_size(AE_SAMPLING_RATE, segment_duration_ms)
    segment_size_ec = calculate_segment_size(EC_SAMPLING_RATE, segment_duration_ms)

    def segment_and_pad(recording, segment_size):
        segments = [recording[i:i+segment_size] for i in range(0, len(recording), segment_size)]
        padded_segments = [np.pad(segment, (0, max(0, segment_size - len(segment))), 'constant', constant_values=pad_value) for segment in segments]
        print(f"Segment lengths: {[len(seg) for seg in padded_segments]}")
        return padded_segments

    ae_segments = segment_and_pad(ae_recording, segment_size_ae)
    ec_segments = segment_and_pad(ec_recording, segment_size_ec)
    return ae_segments, ec_segments

def calculate_features(segment):
    variance = np.var(segment)
    energy = np.sum(np.square(segment))
    return variance, energy

def calculate_features_for_pair(ae_segment, ec_segment):
    ae_variance, ae_energy = calculate_features(ae_segment)
    ec_variance, ec_energy = calculate_features(ec_segment)
    #print(f"Features: AE Variance {ae_variance}, AE Energy {ae_energy}, EC Variance {ec_variance}, EC Energy {ec_energy}")
    return ae_variance, ae_energy, ec_variance, ec_energy

class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d(1, 16, 3, stride=2, padding=1).to(device),
            nn.ReLU().to(device),
            nn.Conv1d(16, 4, 3, stride=2, padding=1).to(device),
            nn.ReLU().to(device),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(4, 16, 3, stride=2, padding=1, output_padding=1).to(device),
            nn.ReLU().to(device),
            nn.ConvTranspose1d(16, 1, 3, stride=2, padding=1, output_padding=1).to(device),
            nn.Sigmoid().to(device),
        )

    def forward(self, x):
        x = x.to(device)
        x = self.encoder(x)
        x = self.decoder(x)
        return x

def extract_features_and_train(train_files, segment_duration_ms):

    autoencoder = ConvAutoencoder().to(device)
    start_time = time.time()  # Record start time

    all_features = []
    for ae_file, ec_file in train_files:
        ae_segments, ec_segments = load_and_segment_recordings(ae_file, ec_file, segment_duration_ms)
        for ae_segment, ec_segment in zip(ae_segments, ec_segments):
            features = calculate_features_for_pair(ae_segment, ec_segment)
            all_features.append(features)
    all_features = np.array(all_features)

    if len(all_features) == 0:
        print("Error: all_features is empty. Please check data extraction.")
        return None, None

    features_tensor = torch.tensor(all_features, dtype=torch.float32).unsqueeze(1)
    features_tensor = features_tensor.to(device)

    if features_tensor.shape[0] == 0 or features_tensor.shape[1] != 1:
        print("Error: features_tensor is incorrectly shaped.")
        return None, None

    print("Shape of features_tensor:", features_tensor.shape)

    autoencoder = ConvAutoencoder()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

    num_epochs = 10
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = autoencoder(features_tensor)
        loss = criterion(outputs, features_tensor)
        loss.backward()
        optimizer.step()

    encoded_features = autoencoder.encoder(features_tensor).detach()

    encoded_features_2d = encoded_features.view(encoded_features.size(0), -1).numpy()

    gmm = GaussianMixture(n_components=3, covariance_type='full', random_state=0).fit(encoded_features_2d)

    # Determine the 97.5th percentile threshold for anomaly detection
    threshold = np.percentile(gmm.score_samples(encoded_features_2d), ANOMALY_THRESHOLD_PERCENTILE)
    joblib.dump((gmm, threshold), 'cae_gmm_model.pkl')

    end_time = time.time()  # Record end time
    print(f"Training time: {end_time - start_time} seconds")  # Print training duration

    return autoencoder, gmm, threshold

def detect_anomalies(autoencoder, gmm, threshold, ae_file, ec_file, segment_duration_ms):
    ae_segments, ec_segments = load_and_segment_recordings(ae_file, ec_file, segment_duration_ms)
    all_features = []

    start_time = time.time()  # Record start time for inference

    for ae_segment, ec_segment in zip(ae_segments, ec_segments):
        features = calculate_features_for_pair(ae_segment, ec_segment)
        all_features.append(features)
    all_features = np.array(all_features)

    end_time = time.time()  # Record end time for inference
    print(f"Inference time: {end_time - start_time} seconds")  # Print inference duration

    if len(all_features) == 0:
        print("Error: all_features is empty. Please check data extraction.")
        return None

    features_tensor = torch.tensor(all_features, dtype=torch.float32).unsqueeze(1)

    with torch.no_grad():
        features_tensor = features_tensor.to(device)
        encoded_features = autoencoder.encoder(features_tensor).detach()
        encoded_features_2d = encoded_features.view(encoded_features.size(0), -1).numpy()

    # Identify anomalous segments
    scores = gmm.score_samples(encoded_features_2d)
    anomalous_segments = scores < threshold

    # Calculate the proportion of anomalous segments
    proportion_anomalous = np.sum(anomalous_segments) / len(anomalous_segments)
    print(f"Proportion of anomalous segments: {proportion_anomalous:.4f}")

    return proportion_anomalous

def flag_recording_as_anomalous(proportion_anomalous, normal_proportions):
    # Determine the threshold for recording-level anomalies
    recording_threshold = np.percentile(normal_proportions, RECORDING_ANOMALY_THRESHOLD)
    is_anomalous = proportion_anomalous > recording_threshold
    return is_anomalous

# Define the directory containing your training data
training_data_directory = "/content/drive/MyDrive/Colab Notebooks/OK_test_6"

# Define the desired segment duration in milliseconds
segment_duration_ms = 50

# Before training loop, move model to GPU
autoencoder = ConvAutoencoder().to(device)

# Load training and testing file paths
train_files = load_training_files(training_data_directory)

# Train the model
autoencoder, gmm, threshold = extract_features_and_train(train_files, segment_duration_ms)

# Calculate the proportion of anomalous segments in the normal training data
normal_proportions = []
for ae_file, ec_file in train_files:
    proportion_anomalous = detect_anomalies(autoencoder, gmm, threshold, ae_file, ec_file, segment_duration_ms)
    normal_proportions.append(proportion_anomalous)

# Save the normal proportions for later comparison
np.save('normal_proportions.npy', normal_proportions)

# Load NOK test files and flag recordings as anomalous if needed
test_data_directory = "/content/drive/MyDrive/Colab Notebooks/NOK_Measurements kopie"

test_files = load_test_files(test_data_directory)
for _, ae_file, ec_file in test_files:
    proportion_anomalous = detect_anomalies(autoencoder, gmm, threshold, ae_file, ec_file, segment_duration_ms)
    is_anomalous = flag_recording_as_anomalous(proportion_anomalous, normal_proportions)
    print(f"Recording {ae_file} is {'anomalous' if is_anomalous else 'normal'}")


Using device: cpu
Train files: [('/content/drive/MyDrive/Colab Notebooks/OK_test_6/2024.02.20_14.28.38_Grinding/raw/Sampling2000KHz_AEKi-0.parquet', '/content/drive/MyDrive/Colab Notebooks/OK_test_6/2024.02.20_14.28.38_Grinding/raw/Sampling100KHz_Irms_Grinding-Grinding spindle current L1-Grinding spindle current L2-Grinding spindle current L3-0.parquet'), ('/content/drive/MyDrive/Colab Notebooks/OK_test_6/2024.02.20_14.43.14_Grinding/raw/Sampling2000KHz_AEKi-0.parquet', '/content/drive/MyDrive/Colab Notebooks/OK_test_6/2024.02.20_14.43.14_Grinding/raw/Sampling100KHz_Irms_Grinding-Grinding spindle current L1-Grinding spindle current L2-Grinding spindle current L3-0.parquet'), ('/content/drive/MyDrive/Colab Notebooks/OK_test_6/2024.02.20_14.42.43_Grinding/raw/Sampling2000KHz_AEKi-0.parquet', '/content/drive/MyDrive/Colab Notebooks/OK_test_6/2024.02.20_14.42.43_Grinding/raw/Sampling100KHz_Irms_Grinding-Grinding spindle current L1-Grinding spindle current L2-Grinding spindle current L3-0.p

UnboundLocalError: local variable 'features_tensor' referenced before assignment